In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dataprep.clean import clean_country
import plotly.express as px

In [4]:
pm25 = pd.read_csv("./dataset/pollution_pm25.csv")
ozone = pd.read_csv("./dataset/pollution_ozone.csv")

In [77]:
pm25.head()

,Exposure Id,Type,Country,ISO3,Region,Name,Exposure Lower,Exposure Mean,Exposure Upper,Year,Pollutant,Pollutant Name,Region Name,Units
0,12391,country,Afghanistan,AFG,country,Afghanistan,21.0,49.3,92.9,1990,pm25,PM25,Country,µg/m3
1,12392,country,Afghanistan,AFG,country,Afghanistan,23.1,48.6,91.9,1995,pm25,PM25,Country,µg/m3
2,12393,country,Afghanistan,AFG,country,Afghanistan,26.5,51.6,92.7,2000,pm25,PM25,Country,µg/m3
3,12394,country,Afghanistan,AFG,country,Afghanistan,26.1,46.2,77.0,2005,pm25,PM25,Country,µg/m3
4,12395,country,Afghanistan,AFG,country,Afghanistan,32.5,51.8,77.9,2010,pm25,PM25,Country,µg/m3


In [78]:
last_5_pm25 = pm25[pm25.Year >= 2010].groupby("ISO3").agg(
        country=("Country", "first"),
        exposure_mean=("Exposure Mean", 'mean'),
        pollutant=("Pollutant", "first"),
        units=("Units", "first"),
).sort_values('exposure_mean', ascending=False)

last_5_pm25 = pd.DataFrame(last_5_pm25.to_records())
last_5_pm25.head()

,ISO3,country,exposure_mean,pollutant,units
0,IND,India,85.64,pm25,µg/m3
1,NPL,Nepal,85.00,pm25,µg/m3
2,QAT,Qatar,79.29,pm25,µg/m3
3,NER,Niger,78.78,pm25,µg/m3
4,EGY,Egypt,70.80,pm25,µg/m3


In [79]:
ozone.head()

,Exposure Id,Type,Country,ISO3,Region,Name,Exposure Lower,Exposure Mean,Exposure Upper,Year,Pollutant,Pollutant Name,Region Name,Units
0,12405,country,Afghanistan,AFG,country,Afghanistan,55.7,56.6,57.5,1990,ozone,Ozone,Country,pbb
1,12406,country,Afghanistan,AFG,country,Afghanistan,53.8,54.7,55.6,1995,ozone,Ozone,Country,pbb
2,12407,country,Afghanistan,AFG,country,Afghanistan,55.3,56.3,57.0,2000,ozone,Ozone,Country,pbb
3,12408,country,Afghanistan,AFG,country,Afghanistan,52.8,53.6,54.5,2005,ozone,Ozone,Country,pbb
4,12409,country,Afghanistan,AFG,country,Afghanistan,52.2,53.0,53.8,2010,ozone,Ozone,Country,pbb


In [80]:
last_5_ozone = ozone[ozone.Year >= 2015].groupby("ISO3").agg(
        country=("Country", "first"),
        exposure_mean=("Exposure Mean", "mean"),
        pollutant=("Pollutant", "first"),
        units=("Units", "first"),
).sort_values('exposure_mean', ascending=False)

last_5_ozone = pd.DataFrame(last_5_ozone.to_records())
last_5_ozone.head()


,ISO3,country,exposure_mean,pollutant,units
0,QAT,Qatar,66.76,ozone,pbb
1,NPL,Nepal,66.02,ozone,pbb
2,IND,India,64.20,ozone,pbb
3,BHR,Bahrain,63.50,ozone,pbb
4,KWT,Kuwait,62.56,ozone,pbb


In [81]:
air_pollution = pd.merge(last_5_pm25, last_5_ozone, on=['ISO3', 'country'], suffixes=['_pm25', '_ozone'])
air_pollution.head()

,ISO3,country,exposure_mean_pm25,pollutant_pm25,units_pm25,exposure_mean_ozone,pollutant_ozone,units_ozone
0,IND,India,85.64,pm25,µg/m3,64.20,ozone,pbb
1,NPL,Nepal,85.00,pm25,µg/m3,66.02,ozone,pbb
2,QAT,Qatar,79.29,pm25,µg/m3,66.76,ozone,pbb
3,NER,Niger,78.78,pm25,µg/m3,45.66,ozone,pbb
4,EGY,Egypt,70.80,pm25,µg/m3,52.88,ozone,pbb


In [82]:
air_pollution.describe()

,exposure_mean_pm25,exposure_mean_ozone
count,204.000000,204.000000
mean,26.895828,40.473922
std,17.747028,10.552569
min,5.544000,13.340000
25%,13.472500,34.525000
50%,21.695000,40.510000
75%,36.510000,47.225000
max,85.640000,66.760000


In [84]:

fig = px.choropleth(
    last_5_pm25,
    locations="ISO3",
    color="exposure_mean",
    hover_name="country",
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# plotly.offline.plot(fig,filename='world.html')
fig.show()

In [85]:
fig = px.bar(last_5_pm25[:20],  x='exposure_mean', y='country', orientation='h')
fig.show()

In [86]:
fig = px.choropleth(
    last_5_ozone,
    locations="ISO3",
    color="exposure_mean",
    hover_name="country",
)
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
# plotly.offline.plot(fig,filename='world.html')
fig.show()

In [87]:
fig = px.bar(last_5_ozone[:20],  x='exposure_mean', y='country', orientation='h')
fig.show()

In [218]:
# some of the most pollutate countries in the world are India, Qatar, Neapl

In [253]:
high_poll = set.intersection(set(last_5_ozone[:20].ISO3), set(last_5_pm25[:20].ISO3))
low_pool = set.union(set(last_5_ozone[-10:].ISO3), set(last_5_pm25[-10:].ISO3))
print(low_pool)
# world largest economies aren't amongt this

{'FSM', 'COK', 'ATG', 'GUM', 'PNG', 'MHL', 'GRL', 'NIU', 'ASM', 'ISL', 'SWE', 'TUV', 'KNA', 'SLB', 'PLW', 'NZL', 'FIN', 'BRB', 'NRU'}


In [247]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

df = air_pollution[air_pollution.ISO3.isin(high_poll)].sort_values(by=['exposure_mean_pm25', 'exposure_mean_ozone'], ascending=False)
fig = go.Figure(data=[go.Table(
    header=dict(values=list(['<b>Country<b><br>World Most Polluted Countries', "PM25 Air Pollution (µg/m3)", "Ozone Air Pollution (pbb)"]),
        line_color='darkslategray',
        fill_color=headerColor,
        align=['left','center'],
        font=dict(color='white')),
    cells=dict(values=[df.country, df.exposure_mean_pm25.round(), df.exposure_mean_ozone.round()],
        line_color='darkslategray',
        # 2-D list of colors for alternating rows
        fill_color = [[rowOddColor,rowEvenColor]*df.shape[0]],
        align = ['left', 'center'],
        font = dict(color = 'darkslategray')
    ))
])

fig.show()


In [221]:
death_risk_file = "/home/amir/big_storage/datasets/IHME-GDB-Risk-all.csv"
death_risk = pd.read_csv(death_risk_file)
death_risk.head()

,measure,location,sex,age,cause,rei,metric,year,val,upper,lower
0,YLDs,Norway,Male,All Ages,Depressive disorders,All risk factors,Number,1992,576.937291,1074.430283,241.176114
1,YLDs,Norway,Female,All Ages,Depressive disorders,All risk factors,Number,1992,1831.591888,3294.474095,665.732721
2,YLDs,Norway,Both,All Ages,Depressive disorders,All risk factors,Number,1992,2408.529178,4122.109930,974.516636
3,YLDs,Norway,Male,All Ages,Depressive disorders,All risk factors,Rate,1992,27.241200,50.731285,11.387593
4,YLDs,Norway,Female,All Ages,Depressive disorders,All risk factors,Rate,1992,84.472852,151.940847,30.703533


In [222]:

death_risk.cause.unique()
causes = [
    "Depressive disorders",
    "Cardiovascular diseases",
    "Chronic respiratory diseases",
    "Neurological disorders",
    "Tracheal, bronchus, and lung cancer",
    "Stomach cancer",
    "Pancreatic cancer",
    "Leukemia",
    "Esophageal cancer",
    "Larynx cancer",
    "Colon and rectum cancer",
    "Liver cancer",
    "Other non-communicable diseases",
]


In [223]:
country_death_risk = death_risk[
    (death_risk.metric == "Rate")
    & (death_risk.sex == "Both")
    & (death_risk.measure == "Deaths")
    & (death_risk.location != "Global")
    & (death_risk.year >= 2010)
    & (death_risk.cause.isin(causes))
]

country_death_risk = clean_country(country_death_risk, 'location', output_format='alpha-3')

air_death_risk = country_death_risk[country_death_risk.rei == "Air pollution"]

all_risk = country_death_risk[country_death_risk.rei == "All risk factors"]


  0%|          | 0/8 [00:00<?, ?it/s]

Country Cleaning Report:
	32970 values cleaned (100.0%)
Result contains 32970 (100.0%) values in the correct format and 0 null values (0.0%)


In [225]:
air_death_all_time = air_death_risk.groupby("location_clean").agg(
    location=("location", "first"),
    val_sum=("val", "sum"),
    val_mean=("val", "mean"),
    val_fisr=("val", "min"),
    val_max=("val", "max"),
).sort_values(by='val_mean', ascending=False).reset_index()
low_air_polution_code = air_death_all_time.location_clean[-10:]
air_death_all_time.head(10)

,location_clean,location,val_sum,val_mean,val_fisr,val_max
0,PRK,Democratic People's Republic of Korea,1910.134131,47.753353,0.003331,123.351110
1,BGR,Bulgaria,1540.176748,38.504419,0.000326,145.560024
2,MKD,North Macedonia,1458.835057,36.470876,0.002041,131.894755
3,SRB,Serbia,1449.701917,36.242548,0.000837,135.474119
4,GEO,Georgia,1292.058295,32.301457,0.000771,135.988400
5,BIH,Bosnia and Herzegovina,1264.110544,31.602764,0.002029,104.551572
6,MMR,Myanmar,1102.766975,27.569174,0.019473,78.154124
7,UKR,Ukraine,1102.508245,27.562706,0.001786,115.002664
8,NPL,Nepal,1091.639518,27.290988,0.006904,67.886708
9,BLR,Belarus,1057.503480,26.437587,0.000930,122.142214


In [226]:
all_risk_all_time = all_risk.groupby("location_clean").agg(
    location=("location", "first"),
    val_sum=("val", "sum"),
    val_mean=("val", "mean"),
    val_fisr=("val", "min"),
    val_max=("val", "max"),
).sort_values(by='val_mean', ascending=False).reset_index()
all_risk_all_time.head(10)

,location_clean,location,val_sum,val_mean,val_fisr,val_max
0,BGR,Bulgaria,11200.610098,93.338417,0.003123,1006.264313
1,UKR,Ukraine,9453.954817,78.782957,0.017578,905.477381
2,LVA,Latvia,8345.684623,69.547372,0.014157,732.975936
3,BLR,Belarus,8294.749733,69.122914,0.008919,792.132002
4,GEO,Georgia,8205.922349,68.382686,0.004716,746.232268
5,SRB,Serbia,7967.470064,66.395584,0.005656,640.609817
6,LTU,Lithuania,7777.899082,64.815826,0.007632,679.502579
7,ROU,Romania,7627.088604,63.559072,0.003360,674.896597
8,HUN,Hungary,7582.500868,63.187507,0.007196,569.864219
9,RUS,Russian Federation,7300.722395,60.839353,0.010103,718.223092


In [254]:

df = pd.merge(all_risk_all_time, air_death_all_time, on=['location', 'location_clean'], suffixes=("_all", "_air"))
df = df[(df.location_clean.isin(list(high_poll))) | (df.location_clean.isin(list(low_pool)))]
df

,location_clean,location,val_sum_all,val_mean_all,val_fisr_all,val_max_all,val_sum_air,val_mean_air,val_fisr_air,val_max_air
21,FIN,Finland,4762.605825,39.688382,0.004967,386.069577,87.911626,2.197791,0.000289,10.048577
28,SWE,Sweden,4204.273950,35.035616,0.010954,324.824395,88.315738,2.207893,0.000812,8.886438
49,NZL,New Zealand,3096.661632,25.805514,0.021584,210.462697,60.363314,1.509083,0.001288,4.096797
70,NPL,Nepal,2264.544549,18.871205,0.025059,128.017735,1091.639518,27.290988,0.006904,67.886708
73,IND,India,2174.296217,18.119135,0.029023,160.538076,911.263207,22.781580,0.007564,54.504756
83,BGD,Bangladesh,1956.903050,16.307525,0.024956,176.612255,819.252552,20.481314,0.006587,65.447012
90,PNG,Papua New Guinea,1829.710992,15.247592,0.062979,128.911282,782.284756,19.557119,0.017181,48.203841
92,AFG,Afghanistan,1777.385480,14.811546,0.088049,169.686960,656.060669,16.401517,0.024083,64.194990
93,PAK,Pakistan,1772.970141,14.774751,0.082929,132.982192,680.474472,17.011862,0.021947,48.482955
113,SAU,Saudi Arabia,1334.358155,11.119651,0.007103,124.888189,447.564440,11.189111,0.001720,43.041053


In [255]:
locations_list = []
for location_clean, locaion in zip(df.location_clean, df.location):
    if location_clean in high_poll:
        locaion = f"<b>{locaion}*</b>"
    locations_list.append(locaion)
print(locations_list)

['Finland', 'Sweden', 'New Zealand', '<b>Nepal*<b>', '<b>India*<b>', '<b>Bangladesh*<b>', 'Papua New Guinea', '<b>Afghanistan*<b>', '<b>Pakistan*<b>', '<b>Saudi Arabia*<b>', '<b>Kuwait*<b>', '<b>Bahrain*<b>', '<b>Qatar*<b>']


In [256]:

fig = go.Figure(data=[
    go.Bar(name='All Risk', x=locations_list, y=df.val_mean_all),
    go.Bar(name='Air Pollution', x=locations_list, y=df.val_mean_air)
])
# Change the bar mode
fig.update_layout(barmode='group',
    xaxis_title="Country (*bold counries have high air pollution)",
    yaxis_title="Value Mean (2010-2015)",
    legend_title="Risk Factor",
    title="Risk of Dying (Comparing Country with clean and not clean air)")
# fig.update_xaxes(categoryorder='total ascending')
fig.show()

In [288]:

death_number = death_risk[
    (death_risk.metric == "Number")
    & (death_risk.sex == "Both")
    & (death_risk.measure == "DALYs")
    & (death_risk.location != "Global")
    & (death_risk.year == 2019)
    & (death_risk.cause.isin(causes))
    & (death_risk.rei=="Air pollution")
]

death_number = clean_country(death_number, 'location', output_format='alpha-3')

death_number.head()

  0%|          | 0/8 [00:00<?, ?it/s]

Country Cleaning Report:
	628 values cleaned (100.0%)
Result contains 628 (100.0%) values in the correct format and 0 null values (0.0%)


,measure,location,sex,age,cause,rei,metric,year,val,upper,lower,location_clean
487943,DALYs,China,Both,All Ages,Cardiovascular diseases,Air pollution,Number,2019,2.612932e+07,3.053316e+07,2.213693e+07,CHN
488559,DALYs,Myanmar,Both,All Ages,Cardiovascular diseases,Air pollution,Number,2019,1.083594e+06,1.311321e+06,8.794192e+05,MMR
488589,DALYs,Philippines,Both,All Ages,"Tracheal, bronchus, and lung cancer",Air pollution,Number,2019,8.193186e+04,1.131842e+05,5.656120e+04,PHL
488757,DALYs,Cambodia,Both,All Ages,Chronic respiratory diseases,Air pollution,Number,2019,4.349903e+04,5.822509e+04,2.966706e+04,KHM
488793,DALYs,Bulgaria,Both,All Ages,"Tracheal, bronchus, and lung cancer",Air pollution,Number,2019,1.822458e+04,2.521782e+04,1.237392e+04,BGR


In [303]:
sum_death_number = death_number.groupby("location_clean").agg(
    val=("val", "sum"),
    location=("location", "first"),
    year=("year", "first"),
).sort_values('val', ascending=False).reset_index()
sum_death_number = sum_death_number[sum_death_number.location_clean.isin(high_poll)]
sum_death_number.head()

,location_clean,val,location,year
1,IND,3.349118e+07,India,2019
3,PAK,4.247819e+06,Pakistan,2019
4,BGD,3.532349e+06,Bangladesh,2019
19,NPL,8.022530e+05,Nepal,2019
23,AFG,7.068587e+05,Afghanistan,2019


In [308]:

headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
    header=dict(values=list(['<b>Country<b>', "Number"]),
        line_color='darkslategray',
        fill_color=headerColor,
        align=['left','center'],
        font=dict(color='white')),
    cells=dict(values=[sum_death_number.location, sum_death_number.val.apply(lambda x: "{:,.0f}".format(x))],
        line_color='darkslategray',
        # 2-D list of colors for alternating rows
        fill_color = [[rowOddColor,rowEvenColor]*sum_death_number.shape[0]],
        align = ['left', 'center'],
        font = dict(color = 'darkslategray')
    ))
])

fig.update_layout(
    title="<b>Disability-adjusted life years (DALYs) for each Country</b><br>calculated as the sum of the years of life lost due to premature mortality (YLLs) from that cause and the years of years of healthy life lost due to disability (YLDs) <br>for people living in states of less than good health resulting from the specific cause.")
# fig.update_xaxes(categoryorder='total ascending')
fig.show()